### 특정 키워드로 word2Vec 단어 유사도 보기

In [1]:
# !pip show gensim

Name: gensim
Version: 3.8.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPLv2.1
Location: /opt/conda/lib/python3.7/site-packages
Requires: smart-open, six, scipy, numpy
Required-by: 


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
df = pd.read_csv('./data/petition.csv', parse_dates=['start', 'end'])

In [6]:
p = r'.*(P2P|은행|금융|주식|증권|공매도).*'
finance = df[df['title'].str.match(p) | df['content'].str.match(p, flags=re.MULTILINE)]
finance.shape

(20619, 8)

In [7]:
finance.head()

,article_id,start,end,answered,votes,category,title,content
7,28,2017-08-19,2017-08-26,0,2137,경제민주화,소액주주를 보호해주십시오,** 존경하옵는 문재인대통령님께\n저는 중국원양자원이라는 KOSPI상장사의 소액 ...
11,32,2017-08-19,2017-09-03,0,30,경제민주화,소액주주 보호를 위해 조사요청 드립니다.,"존경하는 대통령님께,\n코스피 주식종목 중국 원양자원은 현재 계속되는 장** 대표이..."
54,77,2017-08-20,2017-09-04,0,53,교통/건축/국토,"안양시청, 안양지청, 재개발조합 적폐가 심해 수사가 안되고 있습니다. 안양시 지장물...","※지장물관련 국토부의 해석 : 기존 건축물의 철거 공사에 관한 사항의 범위는 전기,..."
68,91,2017-08-20,2017-09-19,0,165,경제민주화,증권 자본시장개정안(상장폐지법 개정안),존경하는 문제인 대통령님께\n먼저 어렵고 힘든 국정운영을 이끄시느라 노고가 많으십니...
100,124,2017-08-20,2017-08-27,0,1,기타,포스코 관련,안녕하십니까?\n최근 뉴스에 주목을 못 받는 뉴스가 있습니다. 아시아경제tv의 포스...


In [8]:
finance.tail()

,article_id,start,end,answered,votes,category,title,content
395468,516999,2019-02-04,2019-03-06,0,2,정치개혁,우리은행채용비리-은행장 출감후보상처벌 및 색출요망!,우리은행채용비리-은행장 출감후보상처벌 및 색출요망!\n우리은행통장 만들려다 취소하고...
395476,517015,2019-02-04,2019-03-06,0,9,육아/교육,◆교육은 개천에서 용이 나올 수 있어야 된다!,◆교육은 개천에서 용이 나올 수 있어야 된다!\n지금 우리나라의 교육현실은 국민의 ...
395477,517018,2019-02-04,2019-03-06,0,3,경제민주화,◆재벌을 근본적으로 개혁하라!,◆재벌을 근본적으로 개혁하라!\n지금 우리나라는 재벌로의 경제력이 집중되어 소득불균...
395495,517046,2019-02-04,2019-03-06,0,2,미래,국민연금 대형 금융사고 !,국민연금 역대 최대의 금융 손실사고 ! 예상된 사고 입니다 !\n작년 1...
395504,517057,2019-02-04,2019-03-06,0,4,기타,안희정을 싫어하며 피해자다움을 강요할 수는 없지만..,안희정을 싫어하며 피해자다움을 강요할 수는 없지만..\n저는 안희정을 좋아하지 않습...


In [9]:
sample_index = 395468

In [10]:
sample_title = finance['title'][sample_index]
sample_title

'우리은행채용비리-은행장 출감후보상처벌 및 색출요망!'

In [11]:
sample_content = finance['content'][sample_index]
sample_content

'우리은행채용비리-은행장 출감후보상처벌 및 색출요망!\\n우리은행통장 만들려다 취소하고 카드도 해지하려합니다.\\n그런데 저만 불편하고 아무것도 달라지는게 없네요\\n방산비리는 사형으로 다스려야 합니다만\\n전쟁이 사실 거의 일어나지 않기에\\n심각성에 비해 방산비리로 인한 폐해는 체감이 그리 크지않습니다.\\n그런데 채용비리는 정부정당과 정치가들의 능력유무를 가르는\\n일자리가 달려있고\\n국민의 최대관심사이자 생계가 달려있습니다.\\n스카이캐슬문제도 결국은 일자리가 그 본질입니다.\\n이 일자리를 가지고 국가의 지원을 받기도 하는 은행에서\\n채용비리가 이루어지고\\n심지어는 이를 감시해야할 금감원등 각종 기관에서\\n채용비리가 저질러진 것입니다.\\n지옥같은 출퇴근과 긴 근로시간, 적시에 쓸수 없는 반쪽짜리 휴가\\n업무와 하등상관없는 장기자랑 체육대회등을 주도하는\\n상사들만 해도 미칠지경인데 그래도 이 경우는 취업이라도 된 경우고\\n채용비리라는 것은 취업자체를 방해하고 속이는 행위입니다.\\n방산비리와는 비교도 안 되게 피해가 직접적이며\\n사회의 신뢰와 화합을 완전히 무너뜨리는 행위입니다.\\n그런데 문재인정부와 사법부는 고작 1년 6개월을 선고했습니다.\\n또한 우리은행장을 심문해서 부정합격자들을 색출하고\\n청탁의뢰를 한 부모나 당사자를 신상공개하고 처벌하면 될 것을\\n여러가지 이유를 갖다 붙여 그냥 놔두고\\n합격자는 못찾겠다는 우리은행말만 닫는 무능의 극치를 달리고 있습니다.\\n한마디로 본인들이 부르짖던 공정사회나 일자리에 관심이나 대비\\n능력이 없는 것입니다.\\n귀걸이사건 때문에 발목을 잡히는 건가요?\\n[요구사항]\\n1. 우리은행장은 1년 6개월을 다 못채울 수도 있고\\n1년 6개월 채운 후 청탁자나 부정합격자의 가족으로부터\\n큰 돈과 노후 또는 다른 일자리를 보장받을 수 있습니다.\\n이렇게 처벌 후 보상이 이루어지는 경우에도\\n뇌물과 청탁관련 법으로 강력처벌하는 제도를 없으면 만들어주십시오\\n앞으로도 또 똑같은 일을

In [18]:
def preprocessing(text):
    text = re.sub('\\\\n', ' ', text)
    
    # 한글, 영문만 남기고 모두 제거
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [19]:
sample_content = preprocessing(sample_content)
sample_content[:1000]

'우리은행채용비리 은행장 출감후보상처벌 및 색출요망  우리은행통장 만들려다 취소하고 카드도 해지하려합니다  그런데 저만 불편하고 아무것도 달라지는게 없네요 방산비리는 사형으로 다스려야 합니다만 전쟁이 사실 거의 일어나지 않기에 심각성에 비해 방산비리로 인한 폐해는 체감이 그리 크지않습니다  그런데 채용비리는 정부정당과 정치가들의 능력유무를 가르는 일자리가 달려있고 국민의 최대관심사이자 생계가 달려있습니다  스카이캐슬문제도 결국은 일자리가 그 본질입니다  이 일자리를 가지고 국가의 지원을 받기도 하는 은행에서 채용비리가 이루어지고 심지어는 이를 감시해야할 금감원등 각종 기관에서 채용비리가 저질러진 것입니다  지옥같은 출퇴근과 긴 근로시간  적시에 쓸수 없는 반쪽짜리 휴가 업무와 하등상관없는 장기자랑 체육대회등을 주도하는 상사들만 해도 미칠지경인데 그래도 이 경우는 취업이라도 된 경우고 채용비리라는 것은 취업자체를 방해하고 속이는 행위입니다  방산비리와는 비교도 안 되게 피해가 직접적이며 사회의 신뢰와 화합을 완전히 무너뜨리는 행위입니다  그런데 문재인정부와 사법부는 고작  년  개월을 선고했습니다  또한 우리은행장을 심문해서 부정합격자들을 색출하고 청탁의뢰를 한 부모나 당사자를 신상공개하고 처벌하면 될 것을 여러가지 이유를 갖다 붙여 그냥 놔두고 합격자는 못찾겠다는 우리은행말만 닫는 무능의 극치를 달리고 있습니다  한마디로 본인들이 부르짖던 공정사회나 일자리에 관심이나 대비 능력이 없는 것입니다  귀걸이사건 때문에 발목을 잡히는 건가요   요구사항     우리은행장은  년  개월을 다 못채울 수도 있고  년  개월 채운 후 청탁자나 부정합격자의 가족으로부터 큰 돈과 노후 또는 다른 일자리를 보장받을 수 있습니다  이렇게 처벌 후 보상이 이루어지는 경우에도 뇌물과 청탁관련 법으로 강력처벌하는 제도를 없으면 만들어주십시오 앞으로도 또 똑같은 일을 눈뜨고 당할 겁니까     우리은행장을 심문하여 부정합격자와 청탁자  부정채용을 도운 인사팀담당자들을 모조리 색출하시고 

In [20]:
%time sentences = finance['content'].apply(preprocessing)

CPU times: user 1.27 s, sys: 1.66 ms, total: 1.27 s
Wall time: 1.29 s


### soynlp를 사용해 토큰화

In [21]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()

In [22]:
sample_title = df['title'][sample_index]
sample_title

'우리은행채용비리-은행장 출감후보상처벌 및 색출요망!'

In [23]:
sample_content = df['content'][sample_index]
sample_content

'우리은행채용비리-은행장 출감후보상처벌 및 색출요망!\\n우리은행통장 만들려다 취소하고 카드도 해지하려합니다.\\n그런데 저만 불편하고 아무것도 달라지는게 없네요\\n방산비리는 사형으로 다스려야 합니다만\\n전쟁이 사실 거의 일어나지 않기에\\n심각성에 비해 방산비리로 인한 폐해는 체감이 그리 크지않습니다.\\n그런데 채용비리는 정부정당과 정치가들의 능력유무를 가르는\\n일자리가 달려있고\\n국민의 최대관심사이자 생계가 달려있습니다.\\n스카이캐슬문제도 결국은 일자리가 그 본질입니다.\\n이 일자리를 가지고 국가의 지원을 받기도 하는 은행에서\\n채용비리가 이루어지고\\n심지어는 이를 감시해야할 금감원등 각종 기관에서\\n채용비리가 저질러진 것입니다.\\n지옥같은 출퇴근과 긴 근로시간, 적시에 쓸수 없는 반쪽짜리 휴가\\n업무와 하등상관없는 장기자랑 체육대회등을 주도하는\\n상사들만 해도 미칠지경인데 그래도 이 경우는 취업이라도 된 경우고\\n채용비리라는 것은 취업자체를 방해하고 속이는 행위입니다.\\n방산비리와는 비교도 안 되게 피해가 직접적이며\\n사회의 신뢰와 화합을 완전히 무너뜨리는 행위입니다.\\n그런데 문재인정부와 사법부는 고작 1년 6개월을 선고했습니다.\\n또한 우리은행장을 심문해서 부정합격자들을 색출하고\\n청탁의뢰를 한 부모나 당사자를 신상공개하고 처벌하면 될 것을\\n여러가지 이유를 갖다 붙여 그냥 놔두고\\n합격자는 못찾겠다는 우리은행말만 닫는 무능의 극치를 달리고 있습니다.\\n한마디로 본인들이 부르짖던 공정사회나 일자리에 관심이나 대비\\n능력이 없는 것입니다.\\n귀걸이사건 때문에 발목을 잡히는 건가요?\\n[요구사항]\\n1. 우리은행장은 1년 6개월을 다 못채울 수도 있고\\n1년 6개월 채운 후 청탁자나 부정합격자의 가족으로부터\\n큰 돈과 노후 또는 다른 일자리를 보장받을 수 있습니다.\\n이렇게 처벌 후 보상이 이루어지는 경우에도\\n뇌물과 청탁관련 법으로 강력처벌하는 제도를 없으면 만들어주십시오\\n앞으로도 또 똑같은 일을

In [24]:
tokened_title = tokenizer.tokenize(sample_title)
tokened_title

['우리은행채용비리', '-', '은행장', '출감후보상처벌', '및', '색출요망', '!']

In [25]:
tokened_content = tokenizer.tokenize(sample_content)
tokened_content[:10]

['우리은행채용비리', '-', '은행장', '출감후보상처벌', '및', '색출요망', '!\\', 'n', '우리은행통장', '만들려다']

In [26]:
print(len(tokened_title))
print(len(tokened_content))

7
367


In [27]:
%time tokens = sentences.apply(tokenizer.tokenize)
tokens[:3]

CPU times: user 19.2 s, sys: 120 ms, total: 19.3 s
Wall time: 19.5 s


7     [존경하옵는, 문재인대통령님께, 저는, 중국원양자원이라는, KOSPI, 상장사의, ...
11    [존경하는, 대통령님께, 코스피, 주식종목, 중국, 원양자원은, 현재, 계속되는, ...
54    [지장물관련, 국토부의, 해석, 기존, 건축물의, 철거, 공사에, 관한, 사항의, ...
Name: content, dtype: object

In [28]:
tokens[sample_index][:10]

['우리은행채용비리',
 '은행장',
 '출감후보상처벌',
 '및',
 '색출요망',
 '우리은행통장',
 '만들려다',
 '취소하고',
 '카드도',
 '해지하려합니다']

In [29]:
# word2vec 모델 학습에 로그를 찍음
import logging
logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

In [30]:
from gensim.models import word2vec

model = word2vec.Word2Vec(tokens, min_count=1)
model

2019-12-31 10:49:32,404 : INFO : 'pattern' package not found; tag filters are not available for English
2019-12-31 10:49:32,408 : INFO : collecting all words and their counts
2019-12-31 10:49:32,410 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-31 10:49:32,868 : INFO : PROGRESS: at sentence #10000, processed 2204292 words, keeping 321082 word types
2019-12-31 10:49:33,433 : INFO : PROGRESS: at sentence #20000, processed 4605574 words, keeping 581661 word types
2019-12-31 10:49:33,481 : INFO : collected 599256 word types from a corpus of 4794874 raw words and 20619 sentences
2019-12-31 10:49:33,483 : INFO : Loading a fresh vocabulary
2019-12-31 10:49:35,197 : INFO : effective_min_count=1 retains 599256 unique words (100% of original 599256, drops 0)
2019-12-31 10:49:35,199 : INFO : effective_min_count=1 leaves 4794874 word corpus (100% of original 4794874, drops 0)
2019-12-31 10:49:36,527 : INFO : deleting the raw counts dictionary of 599256 items
20

In [31]:
model_name = '1minwords'
model.save(model_name)

2019-12-31 10:53:37,825 : INFO : saving Word2Vec object under 1minwords, separately None
2019-12-31 10:53:37,828 : INFO : storing np array 'vectors' to 1minwords.wv.vectors.npy
2019-12-31 10:53:38,062 : INFO : not storing attribute vectors_norm
2019-12-31 10:53:38,064 : INFO : storing np array 'syn1neg' to 1minwords.trainables.syn1neg.npy
2019-12-31 10:53:38,292 : INFO : not storing attribute cum_table
2019-12-31 10:53:39,313 : INFO : saved 1minwords


In [32]:
len(model.wv.vocab)

599256

In [33]:
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:30]

['수',
 '있는',
 '있습니다',
 '그',
 '이',
 '년',
 '합니다',
 '더',
 '하는',
 '하고',
 '대한',
 '및',
 '월',
 '할',
 '제',
 '그리고',
 '한',
 '등',
 '없는',
 '이런',
 '저는',
 '있다',
 '것입니다',
 '일',
 '왜',
 '위해',
 '많은',
 '것은',
 '것이',
 '같은']

In [34]:
from collections import Counter
dict(Counter(vocab).most_common(20))

{'수': <gensim.models.keyedvectors.Vocab at 0x7f2b32013ad0>,
 '있는': <gensim.models.keyedvectors.Vocab at 0x7f2b32013fd0>,
 '있습니다': <gensim.models.keyedvectors.Vocab at 0x7f2b31ff7c10>,
 '그': <gensim.models.keyedvectors.Vocab at 0x7f2b3200efd0>,
 '이': <gensim.models.keyedvectors.Vocab at 0x7f2b320189d0>,
 '년': <gensim.models.keyedvectors.Vocab at 0x7f2b31f9c590>,
 '합니다': <gensim.models.keyedvectors.Vocab at 0x7f2b32011450>,
 '더': <gensim.models.keyedvectors.Vocab at 0x7f2b31fa21d0>,
 '하는': <gensim.models.keyedvectors.Vocab at 0x7f2b32015790>,
 '하고': <gensim.models.keyedvectors.Vocab at 0x7f2b32015d90>,
 '대한': <gensim.models.keyedvectors.Vocab at 0x7f2b3200ef10>,
 '및': <gensim.models.keyedvectors.Vocab at 0x7f2b3200e7d0>,
 '월': <gensim.models.keyedvectors.Vocab at 0x7f2b32018b90>,
 '할': <gensim.models.keyedvectors.Vocab at 0x7f2b320101d0>,
 '제': <gensim.models.keyedvectors.Vocab at 0x7f2b31fb0750>,
 '그리고': <gensim.models.keyedvectors.Vocab at 0x7f2b31f6f750>,
 '한': <gensim.models.keyedvec

In [35]:
min(vocab, key=vocab.get)

'원양자원은'

In [36]:
model.wv['주식']

array([ 0.5348801 , -0.4518868 , -0.8558932 , -2.5102556 ,  1.5505663 ,
        0.5807272 , -0.84905255,  1.062938  , -1.8064356 ,  1.2363285 ,
        0.84620494, -1.962737  , -1.8471124 , -0.48305416,  0.3467479 ,
        0.68996054,  2.6965227 ,  1.184679  , -2.409636  ,  2.3115237 ,
       -0.9800674 ,  0.06898722, -0.66698045,  3.048399  ,  1.691142  ,
       -1.7922671 ,  3.2604172 ,  0.56238276, -2.0309796 ,  4.406316  ,
       -1.7524667 ,  0.32806846,  0.5660751 ,  0.09604076,  0.63435125,
        1.6212549 ,  2.279101  , -0.70745593, -1.0853745 ,  1.2424258 ,
       -1.1193863 ,  1.9145676 ,  0.0273636 , -1.4730723 ,  2.9660323 ,
       -2.1114373 , -0.89676476, -1.6482239 , -0.63613206, -1.5994697 ,
        0.15371688,  1.7742473 , -3.8438544 , -2.1525981 , -0.79838836,
        1.3595701 ,  1.7121974 ,  3.3366435 , -0.1406766 ,  1.410115  ,
        1.611409  ,  3.0250108 , -2.398823  , -3.4449947 , -0.06294995,
        0.8692541 ,  0.9720609 ,  0.6256804 , -1.78764   , -0.66

In [37]:
model.wv.most_similar('주식')

2019-12-31 11:09:46,962 : INFO : precomputing L2-norms of word weight vectors


[('투자', 0.8804969787597656),
 ('외국인', 0.8711764812469482),
 ('빠져나가면', 0.8572873473167419),
 ('어떠케되나', 0.8493990898132324),
 ('코스닥', 0.8457637429237366),
 ('말아먹었을거고', 0.8396831750869751),
 ('작전세력', 0.8366274833679199),
 ('주가', 0.8243718147277832),
 ('외국인만', 0.8212090730667114),
 ('관계들', 0.8207902908325195)]

In [38]:
model.wv.most_similar('부동산')

[('한번예기', 0.8273598551750183),
 ('가격', 0.8251888155937195),
 ('투기', 0.8223367929458618),
 ('빠져야', 0.8173244595527649),
 ('집값', 0.8166038393974304),
 ('박의원', 0.8135038614273071),
 ('폭등으로', 0.808491587638855),
 ('집값을', 0.8079994916915894),
 ('상승을', 0.8052607774734497),
 ('투기조성자', 0.800406813621521)]

In [39]:
# 유사도가 없는 단어 추출
model.wv.doesnt_match('부동산 증권 현금 코스닥 코스피'.split())

'현금'

### 단어 유사도

In [41]:
model.wv.similarity('주식', '부동산')

0.65174824

In [42]:
model.wv.similarity('주식', '투자')

0.88049704

In [44]:
model.wv.similarity('주식', '증권')

0.7964152